In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats

In [435]:
def get_data(filename):
    data = pd.read_csv(filename, names=['Time', 'Voltage'], dtype={"Time": "string", "Voltage": "string"})
    data = data[2:]
    data['Time'] = pd.to_numeric(data['Time'])
    data['Voltage'] = pd.to_numeric(data['Voltage'])
    data['Time'] += abs(data['Time'].iloc[0])
    
    return data

In [436]:
data1 = get_data('data/C/scope_11_1.csv')
data2 = get_data('data/C/scope_11_2.csv')

In [437]:
voltage_diff = data1['Voltage'] - data2['Voltage']
data_diff = pd.DataFrame({'Time': data1['Time'], 'Voltage': voltage_diff})

In [438]:
data_diff_trim = data_diff

In [439]:
# plt.figure(figsize=(10, 6))
# plt.plot(data_diff_trim['Time'], data_diff_trim['Voltage'], marker='', linestyle='-')
# plt.title('Voltage vs. Time')
# plt.xlabel('Time')
# plt.ylabel('Voltage')
# plt.grid(True)
# plt.show()

In [440]:
voltage = np.array(data_diff_trim['Voltage'])
time = np.array(data_diff_trim['Time'])

In [441]:
# voltage thresholds (paper)
#   rising edge = 0.85 V
#   falling edge = 0.2 V
# voltage thresholds (used)
#   rising edge = 1.4 V
#   falling edge = 1.4 V

# TODO: use differnt thresholds to improve accuracy

In [442]:
# convert to square wave
thresh = 1.4
voltage_square = np.where(voltage > thresh, 1, 0)

# plt.figure(figsize=(10, 6))
# plt.plot(data_diff_trim['Time'], data_diff_trim['Voltage'], marker='', linestyle='-')
# plt.plot(time, voltage_square, marker='', linestyle='-')
# plt.title('Voltage vs. Time')
# plt.xlabel('Time')
# plt.ylabel('Voltage')
# plt.grid(True)
# plt.show()

In [443]:
# get indices of bit changes
transition_idx = np.where(np.diff(voltage_square) != 0)[0] + 1

# get time of bit changes
transition_time = time[transition_idx]

# get length of constant bit values
transition_time_diff = np.diff(transition_time)

# split into 0 (dominate) and 1 (recessive) bits
if voltage_square[0] == 0:
    transition_time_diff_1 = transition_time_diff[::2]
    transition_time_diff_0 = transition_time_diff[1::2]
else:
    transition_time_diff_0 = transition_time_diff[::2]
    transition_time_diff_1 = transition_time_diff[1::2]

In [444]:
def prepare_bit_times(transition_time_diff):
    # remove inter-frame gap
    # at most 5 consecutive bits = .000004*5 = .00002
    threshold = 0.000022
    indices = np.where(transition_time_diff > threshold)[0]
    transition_time_diff_trim = list(np.delete(transition_time_diff, indices))
    
    # split repeat bits
    idx = 0
    while (idx < len(transition_time_diff_trim)):
        val = transition_time_diff_trim[idx]
        
        # 1 bit
        # .000004
        if (val < 0.000005):
            idx += 1
            continue
            
        # two bits
        # .000008
        elif (val > 0.000005) and (val < 0.000010):
            new_vals = [val/2, val/2]
            transition_time_diff_trim = transition_time_diff_trim[:idx] + new_vals + transition_time_diff_trim[idx+1:]

            idx += 2
        
        # three bits
        # .000012
        elif (val > 0.000010) and (val < 0.000013):
            new_vals = [val/3, val/3, val/3]
            transition_time_diff_trim = transition_time_diff_trim[:idx] + new_vals + transition_time_diff_trim[idx+1:]

            idx += 3
        
        # four bits
        # .000016
        elif (val > 0.000013) and (val < 0.000018):
            new_vals = [val/4, val/4, val/4, val/4]
            transition_time_diff_trim = transition_time_diff_trim[:idx] + new_vals + transition_time_diff_trim[idx+1:]

            idx += 4
        
        # five bits
        # .000016
        elif (val > 0.000018) and (val < 0.000022):
            new_vals = [val/5, val/5, val/5, val/5, val/5]
            transition_time_diff_trim = transition_time_diff_trim[:idx] + new_vals + transition_time_diff_trim[idx+1:]

            idx += 5
               
        else:
            print(val)
            print("Error")
            
    return transition_time_diff_trim

In [445]:
bit_times_0 = prepare_bit_times(transition_time_diff_0)
bit_times_1 = prepare_bit_times(transition_time_diff_1)

In [446]:
print(min(bit_times_0), max(bit_times_0))
print(min(bit_times_1), max(bit_times_1))

3.993599899998479e-06 4.0704001000035905e-06
3.916799900000045e-06 4.006399949999309e-06


In [470]:
# save data
# np.savetxt('C0_2.txt', bit_times_0)
# np.savetxt('C1_2.txt', bit_times_1)

In [471]:
# first = np.loadtxt('C1_1.txt')
# second = np.loadtxt('C1_2.txt')

# concat = np.concatenate((first, second))
# np.savetxt('C1.txt',concat)

In [23]:
data0 = np.loadtxt('data/bit_times/C0.txt')
data1 = np.loadtxt('data/bit_times/C1.txt')

In [24]:
def extract_features(data, bits_per_data_pt):
    i = 0
    j = bits_per_data_pt
    
    data_features = []
    
    while j < data.shape[0]:
        data_slice = data[i:j]

        mean = np.mean(data_slice)
        std = np.std(data_slice)
        var = np.var(data_slice)
        skew = stats.skew(data_slice)
        kurtosis = stats.kurtosis(data_slice)
        rms = np.sqrt(np.mean(data_slice**2))
        high = np.max(data_slice)
        energy = np.mean(data_slice**2)

        i += bits_per_data_pt
        j += bits_per_data_pt
        
        data_features.append([mean, std, var, skew, kurtosis, rms, high, energy])
    
    return data_features

In [25]:
bits_per_data_pt0 = (data0.shape[0]//140)*5
bits_per_data_pt1 = (data1.shape[0]//140)*5


data0_features = np.array(extract_features(data0, bits_per_data_pt0))
data1_features = np.array(extract_features(data1, bits_per_data_pt1))

In [26]:
data_features = np.concatenate((data0_features, data1_features), axis=1)

In [27]:
np.savetxt('C_features.txt', data_features)

In [38]:
dataA = np.loadtxt('data/features/A_features.txt')
dataB = np.loadtxt('data/features/B_features.txt')
dataC = np.loadtxt('data/features/B_features.txt')

In [50]:
targetA = np.full(dataA.shape[0], 0)
targetB = np.full(dataB.shape[0], 1)
targetC = np.full(dataC.shape[0], 2)

In [51]:
all_data = np.concatenate((dataA, dataB, dataC), axis=0)
all_target = np.concatenate((targetA, targetB, targetC), axis=0)

In [53]:
all_target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [ ]:
# 15730 bits
# 112 bits / message
# 140 frames
# 5 frames per data point
# 28 data points

In [448]:
# bit time approx
# 500*10^3 bits per second
# 2*10^6 seconds per bit
# experimental bit rate is 4*10^6 seconds per bit

In [449]:
# 645,000 sps

In [450]:
# max data points is 2,000,000 (2 Mhz)
# we want 50,000,000 sps
# 0.04 sec to get same sampleing rate

# 1 frame is 441*10^-6 s
# 0.04 sec / 441*10^-6 s ~= 90 frames

# bitrate = 500*10^3 bps
# 1 frame ~80 bits
# 1 frame = 0.00016 seconds = 160 * 10^-6 s

# timeout > .5 ms = 1 ms

# 0.04/10 = 0.004 = 4ms

# 45 frames / 5 = 9 data points

# 1 = LOW
# 2 = HIGH
# diff = 1-2